In [8]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
%matplotlib inline

# annoying warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [9]:
def filtering_features_pipeline(data, target_file, temp_increase_vars):
    '''
    Function to run all of our preprocessing functionalities, with a few different configuration options.
    Will save a new dataframe as csv file in a specified address
    
    INPUTS:
    data: a pandas dataframe containing the raw data
    target_file: a file name and address to save the preprocessed data frames. expects .csv extension
    temp_increase_vars: a boolean value (True or False) on whether or not to create the binary temperature increase variables
    
    OUTPUTS:
    data: a data frame with the preprocessed data
    file saved... the preprocessed data frame saved as a csv file at the specified file extension
    '''
    
    #Add campaign duration and campaignID columns
    #Need to convert column to date_time type
    data['Date.Time'] = pd.to_datetime(data['Date.Time'])

    #Loop through and identify consecutive dates. Create a list of campaign IDs to then create a column from.
    campaign_num = 1
    campaign_id_list = [1]
    for index in range(1,len(data['Date.Time'])):
        if (data['Date.Time'][index] - data['Date.Time'][index-1]).days < 1:
            campaign_id_list.append(campaign_num)
        else:
            campaign_num += 1
            campaign_id_list.append(campaign_num)

    #Create new column of campaign IDs:
    data['CampaignID'] = campaign_id_list

    #Loop through and calculate hours in campaign
    duration_list = [0]

    for index in range(1,len(data['Date.Time'])):
        campaign_num = data['CampaignID'][index]
        campaign_start = min(data['Date.Time'][data['CampaignID'] == campaign_num])
        hrs_since_start = (data['Date.Time'][index] - campaign_start).delta*2.77778e-13
        duration_list.append(hrs_since_start)

    #Create new column for campaign hours
    data['Campaign.Hrs'] = duration_list
    
    dates = data['Date.Time']
    data.drop(['Unnamed: 0', 'Date.Time'], axis = 1, inplace = True)
    
    data.drop(['Pressure'], axis = 1, inplace = True)
    
    #delete rows where Main_Mass_Flow =0
    rows_to_delete = np.where(data.Main_Mass_Flow == 0)[0]
    # also add 337, 2533
    rows_to_delete = np.sort(np.concatenate((rows_to_delete, np.array([337, 2533])), axis = 0))

    data.drop(rows_to_delete, axis = 0, inplace = True)
    
    # replace T_Zone_1 zero vals with median
    new_zero_val_T = np.median(data.T_Zone_1.iloc[np.where(data.T_Zone_1 > 50)])
    data.T_Zone_1.iloc[np.where(data.T_Zone_1 <= 50)] = new_zero_val_T

    # replace Blending vals with median
    new_zero_val_B = np.median(data.Blending.iloc[np.where(data.Blending > 20)])
    data.Blending.iloc[np.where(data.Blending <= 20)] = new_zero_val_B
    
    data = data[data.Main_Mass_Flow > 15000]
    
    if temp_increase_vars:
        for i in range(1,10):
            new_colname = "T_Increase_" + str(i)
            colname_i = "T_Zone_" + str(i)
            colname_inext = "T_Zone_" + str(i+1)
            vals = (data[colname_i] <= data[colname_inext]).astype(int)
            data[new_colname] = vals
   
    data.to_csv(target_file)
    return data

        
    


In [10]:
def scale_training(data, scaler):
    '''
    A function that will peform scaling on training data given that all features are already created.
    
    INPUTS:
    data: a pandas dataframe to be scaled
    scaler: a scaling protocol: "standard" or "minmax"
    
    RETURNS:
    scaler: a scaler object that can be used to scale testing data
    scaled_features: a scaled pandas dataframe object
    '''
    if scaling == "standard":
        scaled_colnames = data.columns
        scaled_features = data.copy()
        features = scaled_features[scaled_colnames]
        scaler = StandardScaler().fit(features.values)
        features = scaler.transform(features.values)
        scaled_features[scaled_colnames] = features
        
    elif scaling == "minmax":
        scaled_colnames = data.columns
        scaled_features = data.copy()
        features = scaled_features[scaled_colnames]
        scaler = MinMaxScaler().fit(features.values)
        features = scaler.transform(features.values)
        scaled_features[scaled_colnames] = features
    
    else:
        raise ValueError(scaler)
        
    return scaler, scaled_features

In [11]:
data = pd.read_csv("../../../datasets/anonymized_SAP_data.csv"")

In [12]:
filtering_features_pipeline(data, './testcsv/with_tempinc.csv', True)

,Main_Mass_Flow,Additive_1_Ratio,Additive_2_Ratio,Additive_3_Ratio,Additive_4_Ratio,Additive_5_Ratio,Additive_6_Ratio,Flow_Gas_Ratio,T_Zone_1,T_Zone_2,...,Campaign.Hrs,T_Increase_1,T_Increase_2,T_Increase_3,T_Increase_4,T_Increase_5,T_Increase_6,T_Increase_7,T_Increase_8,T_Increase_9
0,17947.958984,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.534770,200.130234,187.366165,...,0.000000,0,1,0,1,1,0,1,1,0
1,17942.625000,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.524599,200.140137,187.729889,...,0.966667,0,1,0,1,1,0,1,1,0
2,17955.152344,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.528997,199.947128,187.754242,...,2.116668,0,1,0,1,1,0,1,1,0
3,17965.117188,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.519700,199.723190,187.727036,...,2.766669,0,1,0,0,1,0,1,1,0
4,17949.132812,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.525926,199.627060,187.351547,...,3.750003,0,1,0,0,1,0,1,1,0
5,17958.937500,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.521689,199.841858,187.415497,...,4.666670,0,1,0,1,1,0,1,1,0
6,17957.164062,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.510841,199.586136,187.919907,...,5.400004,0,1,0,1,1,0,1,1,0
7,17964.480469,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.503754,199.705261,187.216400,...,6.650005,0,1,0,1,1,0,1,1,0
8,17945.482422,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.526086,199.523193,187.685547,...,7.550006,0,1,0,1,1,0,1,1,0
9,17957.333984,0.000626,0.053146,0.000876,0.001,0.000910,0.00525,2.524996,198.965714,187.423965,...,8.733340,0,1,0,1,1,0,1,1,0


In [ ]:
data = pd.read_csv("../../../datasets/anonymized_SAP_data.csv")
filtering_features_pipeline(data, './testcsv/no_tempinc.csv', False)